In [1]:
#!pip install duckdb

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
import pandas as pd
import duckdb

In [5]:
sales = pd.read_excel('FL2_DT.xlsx', sheet_name='sales')

In [6]:
sales.head(4)

,ID_ORDER,ID_PRODUCT,ID_BUYER,ID_SELLER,ID_SELLER_COUNTRY,ID_BUYER_COUNTRY,DATE_PAYMENT,BRAND,CATEGORY,REVENUE
0,34033734,13681706,9666775,6723223,203,170,2021-01-07T00:09:24Z,Dsquared2,clothing,57.2746
1,33933770,13575555,8259842,9983902,222,222,2021-01-05T00:23:10Z,Eytys,shoes,168.7399
2,33415990,10239167,8541994,144764,141,204,2021-01-06T00:06:50Z,Dior,shoes,126.4140
3,33415990,10239167,8941774,144764,141,204,2021-01-06T00:06:50Z,Dior,shoes,126.4140


In [7]:
print(sales.dtypes)

ID_ORDER               int64
ID_PRODUCT             int64
ID_BUYER               int64
ID_SELLER              int64
ID_SELLER_COUNTRY      int64
ID_BUYER_COUNTRY       int64
DATE_PAYMENT          object
BRAND                 object
CATEGORY              object
REVENUE              float64
dtype: object


In [8]:
#sales = sales.convert_dtypes()

In [9]:
#print(sales.dtypes)

In [10]:
sales['DATE_PAYMENT']= pd.to_datetime(sales['DATE_PAYMENT'])

In [11]:
print(sales.dtypes)

ID_ORDER                           int64
ID_PRODUCT                         int64
ID_BUYER                           int64
ID_SELLER                          int64
ID_SELLER_COUNTRY                  int64
ID_BUYER_COUNTRY                   int64
DATE_PAYMENT         datetime64[ns, UTC]
BRAND                             object
CATEGORY                          object
REVENUE                          float64
dtype: object


In [12]:
sales.info

<bound method DataFrame.info of       ID_ORDER  ID_PRODUCT  ID_BUYER  ID_SELLER  ID_SELLER_COUNTRY  \
0     34033734    13681706   9666775    6723223                203   
1     33933770    13575555   8259842    9983902                222   
2     33415990    10239167   8541994     144764                141   
3     33415990    10239167   8941774     144764                141   
4     33415990    10239167   8541774     944764                141   
...        ...         ...       ...        ...                ...   
2274  33877283    12184347   8305452   10518837                105   
2275  33877283    12184347   9305452   10518837                105   
2276  34102266    13572750    467486    5678155                105   
2277  34102266    13572750    497446    5679155                105   
2278  34102266    13572750    447486    5679155                105   

      ID_BUYER_COUNTRY              DATE_PAYMENT            BRAND  CATEGORY  \
0                  170 2021-01-07 00:09:24+00:00

In [13]:
country = pd.read_excel('SQL_Test_Datasets.xlsx', sheet_name='country')

In [14]:
country.head(4)

,ID_COUNTRY,REGION,COUNTRY
0,123,EMEA,Lithuania
1,117,EMEA,Latvia
2,67,EMEA,Estonia
3,84,EMEA,Greece


In [15]:
country.info

<bound method DataFrame.info of      ID_COUNTRY REGION     COUNTRY
0           123   EMEA   Lithuania
1           117   EMEA      Latvia
2            67   EMEA     Estonia
3            84   EMEA      Greece
4           175   EMEA     Romania
..          ...    ...         ...
240         127   EMEA  Madagascar
241          63   EMEA       Egypt
242         128   EMEA      Malawi
243         192   EMEA     Somalia
244         225   AMER     Uruguay

[245 rows x 3 columns]>

In [16]:
print(country.dtypes)

ID_COUNTRY     int64
REGION        object
COUNTRY       object
dtype: object


In [17]:
#country = country.convert_dtypes()

In [18]:
#print(country.dtypes)

In [19]:
query1 = "SELECT BRAND, REVENUE FROM sales ORDER BY REVENUE DESC LIMIT 10"

In [20]:
duckdb.query(query1).df()

,BRAND,REVENUE
0,Hermès,9026.6130
1,Hermès,8086.0261
2,Hermès,7113.3345
3,Pomellato,6046.3109
4,Hermès,5397.9300
5,Chanel,5195.4300
6,Chanel,5087.9700
7,Chanel,3633.0968
8,Chanel,2959.3900
9,Louis Vuitton,2210.3732


In [21]:
query2_1 = "SELECT * FROM country JOIN sales ON country.ID_COUNTRY = sales.ID_SELLER_COUNTRY"

In [22]:
query2_2 = """
            WITH revenue_and_items AS
            (SELECT COUNTRY, SUM(REVENUE) as TOTAL_REVENUE, COUNT(ID_SELLER) as ITEM_SOLD
            FROM (SELECT * FROM country JOIN sales ON country.ID_COUNTRY = sales.ID_SELLER_COUNTRY)
            GROUP BY COUNTRY)
            
            SELECT COUNTRY, 
                    ROUND(TOTAL_REVENUE * 100.0/ SUM(TOTAL_REVENUE) OVER(), 2) as PCTG_REVENUE,
                    ROUND(ITEM_SOLD * 100.0/ SUM(ITEM_SOLD) OVER(), 2) as PCTG_ITEMS
            FROM revenue_and_items
            """

In [23]:
duckdb.query(query2_2).df()

,COUNTRY,pctg_revenue,pctg_items
0,Sweden,1.38,1.01
1,United Kingdom,11.40,13.95
2,Monaco,0.69,1.36
3,Romania,0.87,1.01
4,France,23.23,17.42
5,Italy,25.87,33.70
6,Greece,0.43,1.05
7,Germany,6.45,5.79
8,Lithuania,0.42,0.22
9,Spain,5.04,5.09


In [24]:
query3_1 = """SELECT ID_ORDER as ordID, ID_COUNTRY, COUNTRY, ID_SELLER_COUNTRY, REVENUE, ROW_NUMBER (ORDER BY ordID DESC) AS row_index
                    FROM country JOIN sales ON country.ID_COUNTRY = sales.ID_SELLER_COUNTRY
                    """

In [25]:
query3_2 = """SELECT ID_ORDER as ordID, ID_COUNTRY as id_ctry, ID_BUYER_COUNTRY, REVENUE
                    FROM country JOIN sales ON country.ID_COUNTRY = sales.ID_BUYER_COUNTRY"""

In [26]:
query3_3 = """
            WITH sales_country_seller AS
                (SELECT sales.ID_ORDER as ordID, country.COUNTRY as ctry_seller, sales.ID_SELLER_COUNTRY 
                    FROM country JOIN sales ON country.ID_COUNTRY = sales.ID_SELLER_COUNTRY
                    ORDER BY ordID),
            
            sales_country_buyer AS
                (SELECT sales.ID_ORDER as ordID, country.COUNTRY as ctry_buyer, sales.ID_BUYER_COUNTRY, sales.REVENUE
                    FROM country JOIN sales ON country.ID_COUNTRY = sales.ID_BUYER_COUNTRY
                    ORDER BY ordID)
            
            SELECT *
            FROM sales_country_seller
            INNER JOIN sales_country_buyer USING(ordID)"""

In [27]:
duckdb.query(query3_3).df()

,ordid,ctry_seller,ID_SELLER_COUNTRY,ctry_buyer,ID_BUYER_COUNTRY,REVENUE
0,25308441,United Kingdom,222,Canada,38,283.9050
1,29210961,United Arab Emirates,221,Germany,81,138.5910
2,29277765,Sweden,203,United States,223,330.2638
3,29418941,Italy,105,Sweden,203,893.8754
4,29893510,Hong Kong,96,Italy,105,43.9647
...,...,...,...,...,...,...
31084,34505936,Italy,105,Spain,195,27.0000
31085,34505936,Italy,105,Spain,195,27.0000
31086,34505936,Italy,105,Spain,195,27.0000
31087,34505936,Italy,105,Spain,195,27.0000


In [28]:
duckdb.query(query3_2).df()

,ordid,id_ctry,ID_BUYER_COUNTRY,REVENUE
0,34033734,170,170,57.2746
1,33933770,222,222,168.7399
2,33415990,204,204,126.4140
3,33415990,204,204,126.4140
4,33415990,204,204,126.4140
...,...,...,...,...
2274,33877283,103,103,81.0000
2275,33877283,103,103,81.0000
2276,34102266,74,74,83.7000
2277,34102266,74,74,83.7000


In [29]:
duckdb.query(query3_3).df()

,ordid,ctry_seller,ID_SELLER_COUNTRY,ctry_buyer,ID_BUYER_COUNTRY,REVENUE
0,25308441,United Kingdom,222,Canada,38,283.9050
1,29210961,United Arab Emirates,221,Germany,81,138.5910
2,29277765,Sweden,203,United States,223,330.2638
3,29418941,Italy,105,Sweden,203,893.8754
4,29893510,Hong Kong,96,Italy,105,43.9647
...,...,...,...,...,...,...
31084,34505936,Italy,105,Spain,195,27.0000
31085,34505936,Italy,105,Spain,195,27.0000
31086,34505936,Italy,105,Spain,195,27.0000
31087,34505936,Italy,105,Spain,195,27.0000


In [30]:
query5_1 = """WITH buyers_week2 AS
                    (SELECT DISTINCT ID_BUYER, DATE_PAYMENT, COUNT(ID_PRODUCT) as nb_item
                    FROM sales
                    GROUP BY ID_BUYER, DATE_PAYMENT
                    HAVING DATE_PAYMENT >= '2021-01-08' AND DATE_PAYMENT <= '2021-01-16'
                    ORDER BY DATE_PAYMENT ASC),
                    
                buyers_week1 AS
                    (SELECT DISTINCT ID_BUYER, DATE_PAYMENT, COUNT(ID_PRODUCT) as nb_item
                    FROM sales
                    GROUP BY ID_BUYER, DATE_PAYMENT
                    HAVING DATE_PAYMENT >= '2021-01-01' AND DATE_PAYMENT <= '2021-01-08' and nb_item >= 1
                    ORDER BY DATE_PAYMENT ASC)
                    
                SELECT ID_BUYER
                FROM (buyers_week2 INTERSECT buyers_week1) as """

In [31]:
duckdb.query(query5_1).df()

RuntimeError: Parser Error: syntax error at or near "INTERSECT"
LINE 16:                 FROM (buyers_week2 INTERSECT buyers_week1) as 
                                            ^

In [32]:
query5_2 = """SELECT DISTINCT ID_BUYER, DATE_PAYMENT
                    FROM sales
                    WHERE DATE_PAYMENT >= '2021-01-08' AND DATE_PAYMENT <= '2021-01-16'
                    
                    INTERSECT
                    
                    SELECT DISTINCT ID_BUYER, DATE_PAYMENT
                    FROM sales
                    WHERE DATE_PAYMENT >= '2021-01-01' AND DATE_PAYMENT <= '2021-01-08'
                    """

In [33]:
duckdb.query(query5_2).df()

,ID_BUYER,DATE_PAYMENT
